# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 2.2.2
- **Date :** 10/11/2023
- **Heure :** 22:20
- **Données :** `ready_Evaporation_Sunshine.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Omar/ready_Evaporation_Sunshine.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,2008,12,1,0,3.141593,3.141593,2.748894,-36.080477,146.916280,0
2008-12-01,5.8,17.6,3.2,52.0,26.0,30.0,71.0,37.0,1008.2,1007.5,...,2008,12,1,18,3.141593,3.141593,2.748894,-37.562303,143.860565,2
2008-12-02,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,2008,12,2,0,2.748894,1.963495,3.534292,-36.080477,146.916280,0
2008-12-02,4.3,19.5,0.4,44.0,20.0,24.0,66.0,47.0,1009.3,1007.3,...,2008,12,2,18,2.748894,2.748894,2.748894,-37.562303,143.860565,2
2008-12-03,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,2008,12,3,0,3.534292,3.141593,3.534292,-36.080477,146.916280,0


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10453 entries, 2008-12-01 to 2017-06-25
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         10453 non-null  float64
 1   MaxTemp         10453 non-null  float64
 2   Rainfall        10453 non-null  float64
 3   WindGustSpeed   10453 non-null  float64
 4   WindSpeed9am    10453 non-null  float64
 5   WindSpeed3pm    10453 non-null  float64
 6   Humidity9am     10453 non-null  float64
 7   Humidity3pm     10453 non-null  float64
 8   Pressure9am     10453 non-null  float64
 9   Pressure3pm     10453 non-null  float64
 10  Cloud9am        10453 non-null  float64
 11  Cloud3pm        10453 non-null  float64
 12  Temp9am         10453 non-null  float64
 13  Temp3pm         10453 non-null  float64
 14  RainToday       10453 non-null  int64  
 15  RainTomorrow    10453 non-null  int64  
 16  year            10453 non-null  int64  
 17  month           10453 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[ 0,  7],
         [20, 23],
         [ 5,  8],
         [ 4, 17],
         [ 5, 21]]),
  array([[ 4, 13],
         [21, 24],
         [17, 20],
         [10, 16],
         [ 9, 19]]),
  array([[ 7, 11],
         [16, 23],
         [14, 20],
         [18, 22],
         [ 3, 20]]),
  array([[ 3, 11],
         [ 9, 13],
         [14, 23],
         [21, 24],
         [19, 22]]),
  array([[20, 23],
         [15, 18],
         [ 6,  9],
         [11, 19],
         [14, 17]]),
  array([[13, 17],
         [21, 24],
         [ 9, 22],
         [21, 24],
         [ 5, 17]]),
  array([[21, 24],
         [ 8, 12],
         [ 3, 23],
         [16, 23],
         [12, 16]]),
  array([[ 8, 19],
         [18, 22],
         [14, 23],
         [17, 20],
         [16, 19]]),
  array([[ 2, 15],
         [ 1,  7],
         [ 9, 19],
         [ 6, 13],
         [10, 13]]),
  array([[ 6,  9],
         [ 2, 13],
         [ 4, 13],
         [ 1,  4],
         [1

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1322,54
1,201,165


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1376
           1       0.75      0.45      0.56       366

    accuracy                           0.85      1742
   macro avg       0.81      0.71      0.74      1742
weighted avg       0.84      0.85      0.84      1742

